# Supermind Data Science Assignment  : Crypto Detection 

### Name : Devang Papinwar
### Email : papinwardevang@gmail.com
### Contact No : 7420039018

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import contractions
from math import sqrt, pow, exp
from sklearn.feature_extraction.text import CountVectorizer
sns.set()
import spacy
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

/opt/anaconda3/lib/python3.9/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


### Preparing Dataset

In [2]:
definition = pd.read_csv('term_def.csv')
definition

,terms,definition1,definition2
0,51% attack,A hypothetical situation where more than half ...,NaN
1,51% attack protection,A protection mechanism implemented by several ...,NaN
2,AFK,Away From Keyboard; used on social media platf...,NaN
3,Airdrop,An event where a blockchain project distribute...,NaN
4,Altcoin,Any cryptocurrency that is an alternative to B...,NaN
...,...,...,...
155,Vyper,A Python-like programming language for the Eth...,NaN
156,Wallet (Cold),A wallet disconnected from the internet.,NaN
157,Wallet (Hot),A wallet connected to the Internet.,NaN
158,Wallet (Multisignature),A wallet that requires multiple digital signat...,NaN


In [3]:
data = pd.DataFrame()
data['sentence'] = definition['definition1']
data

,sentence
0,A hypothetical situation where more than half ...
1,A protection mechanism implemented by several ...
2,Away From Keyboard; used on social media platf...
3,An event where a blockchain project distribute...
4,Any cryptocurrency that is an alternative to B...
...,...
155,A Python-like programming language for the Eth...
156,A wallet disconnected from the internet.
157,A wallet connected to the Internet.
158,A wallet that requires multiple digital signat...


### Preprocess the data

In [4]:
def text_preprocessing(text):
    # Removing contents of tags and all for further text processing
    data['sentence'].replace(regex=True,inplace=True, to_replace= r'<.+?>', value=r' ')
    
    # Removing the \t charecter from the descriptions field
    data['sentence'] = data['sentence'].str.replace(r'\n', ' ')
    
    data['sentence'] = data['sentence'].str.replace(r'  ', '')

    # Removing links from all for further text processing
    data['sentence'].replace(regex=True,inplace=True, to_replace= r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', value=r' ')
    
    # Remove punctuation
    data['sentence'].replace(regex=True,inplace=True, to_replace= r'[^\w\d\s]', value=r'')
    
    # Removing rows with empty columns 
    data.dropna(subset=['sentence'],inplace=True)
    
    return data

In [5]:
data = text_preprocessing(data)
data

/var/folders/23/12jln18952d9zg137wpr4bq80000gn/T/ipykernel_26778/4089952422.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data['sentence'] = data['sentence'].str.replace(r'\n', ' ')


,sentence
0,A hypothetical situation where more than half ...
1,A protection mechanism implemented by several ...
2,Away From Keyboard used on social media platfo...
3,An event where a blockchain project distribute...
4,Any cryptocurrency that is an alternative to B...
...,...
155,A Pythonlike programming language for the Ethe...
156,A wallet disconnected from the internet
157,A wallet connected to the Internet
158,A wallet that requires multiple digital signat...


### Preprocess the Data removing stopwords , tokenization , lowering charecters

In [6]:
# spaCy model:
nlp = en_core_web_md.load()

# Tags to remove from the text
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
tokens = []
for summary in nlp.pipe(data['sentence']):
   proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
   tokens.append(proj_tok)
tokens

[['hypothetical',
  'situation',
  'half',
  'computing',
  'power',
  'blockchain',
  'network',
  'control',
  'person',
  'group',
  'allow',
  'dictate',
  'transaction',
  'verify',
  'allow',
  'prevent',
  'user',
  'complete',
  'confirm',
  'transaction',
  'cause',
  'havoc',
  'system',
  'doublespend',
  'coin'],
 ['protection',
  'mechanism',
  'implement',
  'cryptocurrencie',
  'require',
  'total',
  'hashing',
  'power',
  'work',
  'entity',
  'difficult',
  'attacker',
  'd',
  'need',
  'resource',
  'time',
  'threshold',
  'have',
  'additional',
  'safeguard',
  'feature',
  'agree',
  'transaction',
  'send',
  'make',
  'unable',
  'doublespend',
  'notice',
  'change',
  'chain'],
 ['keyboard',
  'social',
  'medium',
  'platform',
  'like',
  'twitter',
  'user',
  'share',
  'trading',
  'activity',
  'want',
  'receive',
  'message',
  'log',
  'account',
  'thing',
  'afk',
  'trade',
  'extended',
  'period',
  'time',
  'active',
  'feed'],
 ['event',
  

### Create a new column for tokens

In [7]:
data["tokens"] = tokens
data

,sentence,tokens
0,A hypothetical situation where more than half ...,"[hypothetical, situation, half, computing, pow..."
1,A protection mechanism implemented by several ...,"[protection, mechanism, implement, cryptocurre..."
2,Away From Keyboard used on social media platfo...,"[keyboard, social, medium, platform, like, twi..."
3,An event where a blockchain project distribute...,"[event, blockchain, project, distribute, free,..."
4,Any cryptocurrency that is an alternative to B...,"[cryptocurrency, alternative, bitcoin]"
...,...,...
155,A Pythonlike programming language for the Ethe...,"[pythonlike, programming, language, ethereum, ..."
156,A wallet disconnected from the internet,"[wallet, disconnect, internet]"
157,A wallet connected to the Internet,"[wallet, connect, internet]"
158,A wallet that requires multiple digital signat...,"[wallet, require, multiple, digital, signature..."


### Create a Dictionary from the Bag of Words

In [8]:
dictionary = Dictionary(data['tokens'])
print(dictionary.token2id)

{'allow': 0, 'blockchain': 1, 'cause': 2, 'coin': 3, 'complete': 4, 'computing': 5, 'confirm': 6, 'control': 7, 'dictate': 8, 'doublespend': 9, 'group': 10, 'half': 11, 'havoc': 12, 'hypothetical': 13, 'network': 14, 'person': 15, 'power': 16, 'prevent': 17, 'situation': 18, 'system': 19, 'transaction': 20, 'user': 21, 'verify': 22, 'additional': 23, 'agree': 24, 'attacker': 25, 'chain': 26, 'change': 27, 'cryptocurrencie': 28, 'd': 29, 'difficult': 30, 'entity': 31, 'feature': 32, 'hashing': 33, 'have': 34, 'implement': 35, 'make': 36, 'mechanism': 37, 'need': 38, 'notice': 39, 'protection': 40, 'require': 41, 'resource': 42, 'safeguard': 43, 'send': 44, 'threshold': 45, 'time': 46, 'total': 47, 'unable': 48, 'work': 49, 'account': 50, 'active': 51, 'activity': 52, 'afk': 53, 'extended': 54, 'feed': 55, 'keyboard': 56, 'like': 57, 'log': 58, 'medium': 59, 'message': 60, 'period': 61, 'platform': 62, 'receive': 63, 'share': 64, 'social': 65, 'thing': 66, 'trade': 67, 'trading': 68, 'tw

In [9]:
# Generate a corpus from the BagofWords Dictionary
corpus = [dictionary.doc2bow(doc) for doc in data['tokens']]
corpus

[[(0, 2),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1)],
 [(9, 1),
  (16, 1),
  (20, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1)],
 [(21, 1),
  (46, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1)],
 [(1, 1), (3, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1)],
 [(77, 1), (78, 1), (79, 1)],
 [(57, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (

### Create a Model for topic modeling ie. Latent Dirichlet Allocation

In [10]:
# Train the unsupervised machine learning model on the data
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, num_topics=10, workers = 4, passes=10)

In [11]:
# Print the topics modeled from the data
lda_model.print_topics(0)

[(3,
  '0.021*"currency" + 0.021*"burn" + 0.017*"key" + 0.016*"miner" + 0.013*"message" + 0.009*"private" + 0.009*"market" + 0.009*"datum" + 0.009*"database" + 0.009*"indicator"'),
 (7,
  '0.020*"price" + 0.020*"network" + 0.014*"blockchain" + 0.011*"transaction" + 0.011*"mining" + 0.009*"hold" + 0.009*"block" + 0.009*"computer" + 0.009*"asset" + 0.009*"crypto"'),
 (4,
  '0.018*"bitcoin" + 0.011*"transaction" + 0.011*"protocol" + 0.010*"example" + 0.010*"item" + 0.008*"create" + 0.008*"blockchain" + 0.008*"process" + 0.008*"security" + 0.008*"record"'),
 (5,
  '0.025*"blockchain" + 0.017*"system" + 0.017*"coin" + 0.014*"network" + 0.014*"transaction" + 0.010*"value" + 0.010*"high" + 0.010*"computer" + 0.010*"service" + 0.010*"control"'),
 (1,
  '0.028*"blockchain" + 0.018*"block" + 0.018*"miner" + 0.016*"transaction" + 0.011*"process" + 0.010*"datum" + 0.010*"stake" + 0.009*"work" + 0.009*"system" + 0.009*"exchange"'),
 (9,
  '0.016*"private" + 0.015*"key" + 0.013*"node" + 0.013*"hash"

In [12]:
lda_model[corpus][0]

[(5, 0.96537995)]

In [13]:
# Visualising the topics modeled from the data
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: t

### Create a Bag of Words from the Latent Dirichlet Allocation's modeled topics

In [14]:
# For each topic modeled show the high probability words
for idx, topic in lda_model.print_topics(0):
   print('Topic: {} \nWords: {}'.format(idx, topic))

# Genarate the number of sub topics for main topics generated
topics = lda_model.print_topics(idx, 20)
topics

Topic: 9 
Words: 0.016*"private" + 0.015*"key" + 0.013*"node" + 0.013*"hash" + 0.013*"label" + 0.012*"cryptocurrencie" + 0.012*"access" + 0.012*"ledger" + 0.012*"product" + 0.011*"price"
Topic: 2 
Words: 0.022*"system" + 0.016*"digital" + 0.015*"token" + 0.014*"like" + 0.013*"coin" + 0.013*"transaction" + 0.013*"blockchain" + 0.010*"property" + 0.010*"exchange" + 0.010*"nonfungible"
Topic: 4 
Words: 0.018*"bitcoin" + 0.011*"transaction" + 0.011*"protocol" + 0.010*"example" + 0.010*"item" + 0.008*"create" + 0.008*"blockchain" + 0.008*"process" + 0.008*"security" + 0.008*"record"
Topic: 3 
Words: 0.021*"currency" + 0.021*"burn" + 0.017*"key" + 0.016*"miner" + 0.013*"message" + 0.009*"private" + 0.009*"market" + 0.009*"datum" + 0.009*"database" + 0.009*"indicator"
Topic: 0 
Words: 0.024*"blockchain" + 0.021*"application" + 0.018*"transaction" + 0.014*"information" + 0.014*"investor" + 0.014*"digital" + 0.011*"identity" + 0.011*"require" + 0.011*"project" + 0.011*"central"
Topic: 7 
Words:

[(9,
  '0.016*"private" + 0.015*"key" + 0.013*"node" + 0.013*"hash" + 0.013*"label" + 0.012*"cryptocurrencie" + 0.012*"access" + 0.012*"ledger" + 0.012*"product" + 0.011*"price" + 0.011*"hold" + 0.011*"buy" + 0.008*"computer" + 0.008*"user" + 0.008*"process" + 0.008*"sell" + 0.008*"customer" + 0.008*"tree" + 0.008*"wallet" + 0.008*"trade"'),
 (6,
  '0.015*"blockchain" + 0.009*"market" + 0.009*"crypto" + 0.009*"user" + 0.009*"transaction" + 0.009*"like" + 0.009*"investor" + 0.009*"value" + 0.009*"cryptocurrency" + 0.009*"decentralised" + 0.009*"cryptocurrencie" + 0.009*"smart" + 0.007*"contract" + 0.007*"asset" + 0.006*"block" + 0.006*"network" + 0.006*"chain" + 0.006*"condition" + 0.006*"trading" + 0.006*"link"'),
 (5,
  '0.025*"blockchain" + 0.017*"system" + 0.017*"coin" + 0.014*"network" + 0.014*"transaction" + 0.010*"value" + 0.010*"high" + 0.010*"computer" + 0.010*"service" + 0.010*"control" + 0.010*"fear" + 0.007*"allow" + 0.007*"power" + 0.007*"connect" + 0.007*"activity" + 0.007

### Clean the Bag of Words

In [15]:
tapics = []
for idx , topic in topics:
    tapics.append(topic.split('+'))
tapics

[['0.016*"private" ',
  ' 0.015*"key" ',
  ' 0.013*"node" ',
  ' 0.013*"hash" ',
  ' 0.013*"label" ',
  ' 0.012*"cryptocurrencie" ',
  ' 0.012*"access" ',
  ' 0.012*"ledger" ',
  ' 0.012*"product" ',
  ' 0.011*"price" ',
  ' 0.011*"hold" ',
  ' 0.011*"buy" ',
  ' 0.008*"computer" ',
  ' 0.008*"user" ',
  ' 0.008*"process" ',
  ' 0.008*"sell" ',
  ' 0.008*"customer" ',
  ' 0.008*"tree" ',
  ' 0.008*"wallet" ',
  ' 0.008*"trade"'],
 ['0.015*"blockchain" ',
  ' 0.009*"market" ',
  ' 0.009*"crypto" ',
  ' 0.009*"user" ',
  ' 0.009*"transaction" ',
  ' 0.009*"like" ',
  ' 0.009*"investor" ',
  ' 0.009*"value" ',
  ' 0.009*"cryptocurrency" ',
  ' 0.009*"decentralised" ',
  ' 0.009*"cryptocurrencie" ',
  ' 0.009*"smart" ',
  ' 0.007*"contract" ',
  ' 0.007*"asset" ',
  ' 0.006*"block" ',
  ' 0.006*"network" ',
  ' 0.006*"chain" ',
  ' 0.006*"condition" ',
  ' 0.006*"trading" ',
  ' 0.006*"link"'],
 ['0.025*"blockchain" ',
  ' 0.017*"system" ',
  ' 0.017*"coin" ',
  ' 0.014*"network" ',
  ' 0.

In [16]:
tipics = []
for tapic in tapics:
    for tap in tapic:
        tipics.append(tap.split('*'))
tipics

[['0.016', '"private" '],
 [' 0.015', '"key" '],
 [' 0.013', '"node" '],
 [' 0.013', '"hash" '],
 [' 0.013', '"label" '],
 [' 0.012', '"cryptocurrencie" '],
 [' 0.012', '"access" '],
 [' 0.012', '"ledger" '],
 [' 0.012', '"product" '],
 [' 0.011', '"price" '],
 [' 0.011', '"hold" '],
 [' 0.011', '"buy" '],
 [' 0.008', '"computer" '],
 [' 0.008', '"user" '],
 [' 0.008', '"process" '],
 [' 0.008', '"sell" '],
 [' 0.008', '"customer" '],
 [' 0.008', '"tree" '],
 [' 0.008', '"wallet" '],
 [' 0.008', '"trade"'],
 ['0.015', '"blockchain" '],
 [' 0.009', '"market" '],
 [' 0.009', '"crypto" '],
 [' 0.009', '"user" '],
 [' 0.009', '"transaction" '],
 [' 0.009', '"like" '],
 [' 0.009', '"investor" '],
 [' 0.009', '"value" '],
 [' 0.009', '"cryptocurrency" '],
 [' 0.009', '"decentralised" '],
 [' 0.009', '"cryptocurrencie" '],
 [' 0.009', '"smart" '],
 [' 0.007', '"contract" '],
 [' 0.007', '"asset" '],
 [' 0.006', '"block" '],
 [' 0.006', '"network" '],
 [' 0.006', '"chain" '],
 [' 0.006', '"con

In [17]:
tops = []

for tip in tipics:   
    tops.append(tip[1][1:-2])
tops


['private',
 'key',
 'node',
 'hash',
 'label',
 'cryptocurrencie',
 'access',
 'ledger',
 'product',
 'price',
 'hold',
 'buy',
 'computer',
 'user',
 'process',
 'sell',
 'customer',
 'tree',
 'wallet',
 'trad',
 'blockchain',
 'market',
 'crypto',
 'user',
 'transaction',
 'like',
 'investor',
 'value',
 'cryptocurrency',
 'decentralised',
 'cryptocurrencie',
 'smart',
 'contract',
 'asset',
 'block',
 'network',
 'chain',
 'condition',
 'trading',
 'lin',
 'blockchain',
 'system',
 'coin',
 'network',
 'transaction',
 'value',
 'high',
 'computer',
 'service',
 'control',
 'fear',
 'allow',
 'power',
 'connect',
 'activity',
 'internet',
 'entire',
 'product',
 'history',
 'walle',
 'blockchain',
 'application',
 'transaction',
 'information',
 'investor',
 'digital',
 'identity',
 'require',
 'project',
 'central',
 'device',
 'mining',
 'network',
 'cryptocurrency',
 'record',
 'multiple',
 'entity',
 'system',
 'person',
 'contro',
 'token',
 'transaction',
 'wallet',
 'coin',
 

### Getting the terms from the abbrevations part

In [18]:
terms = []

for term in definition['terms']:
    terms.append(term.split(' '))
terms

[['51%', 'attack'],
 ['51%', 'attack', 'protection'],
 ['AFK'],
 ['Airdrop'],
 ['Altcoin'],
 ['AML'],
 ['ATH'],
 ['Bearish'],
 ['Bear', 'market'],
 ['Bitcoin'],
 ['Blockchain'],
 ['Block', 'height'],
 ['Block', 'reward'],
 ['Bollinger', 'bands'],
 ['BTD'],
 ['Bullish'],
 ['Casper'],
 ['Centralised'],
 ['Coinless', 'protocol'],
 ['Chaffing'],
 ['Confirmation'],
 ['Crowdsale'],
 ['Cryptoart'],
 ['Cryptocurrency'],
 ['Crypto', 'derivatives'],
 ['Cryptoeconomics'],
 ['Cryptography'],
 ['Decentralised'],
 ['Distribution'],
 ['Distributed', 'ledger'],
 ['Digital', 'gold'],
 ['Dumping'],
 ['DYOR'],
 ['Entry', 'and', 'exit', 'points'],
 ['Etherscan'],
 ['Fear', 'and', 'greed', 'index'],
 ['Fiat', 'gateways'],
 ['Flippening'],
 ['FOMO'],
 ['Fork'],
 ['FUD'],
 ['FUDster'],
 ['Futures'],
 ['Gas'],
 ['Halving'],
 ['Hash', 'function'],
 ['Hedging'],
 ['HODL'],
 ['Hot', 'wallet'],
 ['IDO'],
 ['IEO'],
 ['Inflation'],
 ['KYC'],
 ['Lambo'],
 ['Memecoin'],
 ['Mimblewimble'],
 ['Miner'],
 ['Mining'],
 ['

In [19]:
terms1 = []
for term in terms:
    for t in term:
        terms1.append(t.lower())
        
terms1

['51%',
 'attack',
 '51%',
 'attack',
 'protection',
 'afk',
 'airdrop',
 'altcoin',
 'aml',
 'ath',
 'bearish',
 'bear',
 'market',
 'bitcoin',
 'blockchain',
 'block',
 'height',
 'block',
 'reward',
 'bollinger',
 'bands',
 'btd',
 'bullish',
 'casper',
 'centralised',
 'coinless',
 'protocol',
 'chaffing',
 'confirmation',
 'crowdsale',
 'cryptoart',
 'cryptocurrency',
 'crypto',
 'derivatives',
 'cryptoeconomics',
 'cryptography',
 'decentralised',
 'distribution',
 'distributed',
 'ledger',
 'digital',
 'gold',
 'dumping',
 'dyor',
 'entry',
 'and',
 'exit',
 'points',
 'etherscan',
 'fear',
 'and',
 'greed',
 'index',
 'fiat',
 'gateways',
 'flippening',
 'fomo',
 'fork',
 'fud',
 'fudster',
 'futures',
 'gas',
 'halving',
 'hash',
 'function',
 'hedging',
 'hodl',
 'hot',
 'wallet',
 'ido',
 'ieo',
 'inflation',
 'kyc',
 'lambo',
 'memecoin',
 'mimblewimble',
 'miner',
 'mining',
 'mining',
 'difficulty',
 'mining',
 'rigs',
 'moon',
 'nft',
 'node',
 'peer',
 'to',
 'peer',
 '

### Adding them to the Final BagOfWords

In [20]:
data = tops.extend(terms1)

for term in terms1:
    tops.append(term)

tops

['private',
 'key',
 'node',
 'hash',
 'label',
 'cryptocurrencie',
 'access',
 'ledger',
 'product',
 'price',
 'hold',
 'buy',
 'computer',
 'user',
 'process',
 'sell',
 'customer',
 'tree',
 'wallet',
 'trad',
 'blockchain',
 'market',
 'crypto',
 'user',
 'transaction',
 'like',
 'investor',
 'value',
 'cryptocurrency',
 'decentralised',
 'cryptocurrencie',
 'smart',
 'contract',
 'asset',
 'block',
 'network',
 'chain',
 'condition',
 'trading',
 'lin',
 'blockchain',
 'system',
 'coin',
 'network',
 'transaction',
 'value',
 'high',
 'computer',
 'service',
 'control',
 'fear',
 'allow',
 'power',
 'connect',
 'activity',
 'internet',
 'entire',
 'product',
 'history',
 'walle',
 'blockchain',
 'application',
 'transaction',
 'information',
 'investor',
 'digital',
 'identity',
 'require',
 'project',
 'central',
 'device',
 'mining',
 'network',
 'cryptocurrency',
 'record',
 'multiple',
 'entity',
 'system',
 'person',
 'contro',
 'token',
 'transaction',
 'wallet',
 'coin',
 

### Final Bag of Words

In [21]:
tops

['private',
 'key',
 'node',
 'hash',
 'label',
 'cryptocurrencie',
 'access',
 'ledger',
 'product',
 'price',
 'hold',
 'buy',
 'computer',
 'user',
 'process',
 'sell',
 'customer',
 'tree',
 'wallet',
 'trad',
 'blockchain',
 'market',
 'crypto',
 'user',
 'transaction',
 'like',
 'investor',
 'value',
 'cryptocurrency',
 'decentralised',
 'cryptocurrencie',
 'smart',
 'contract',
 'asset',
 'block',
 'network',
 'chain',
 'condition',
 'trading',
 'lin',
 'blockchain',
 'system',
 'coin',
 'network',
 'transaction',
 'value',
 'high',
 'computer',
 'service',
 'control',
 'fear',
 'allow',
 'power',
 'connect',
 'activity',
 'internet',
 'entire',
 'product',
 'history',
 'walle',
 'blockchain',
 'application',
 'transaction',
 'information',
 'investor',
 'digital',
 'identity',
 'require',
 'project',
 'central',
 'device',
 'mining',
 'network',
 'cryptocurrency',
 'record',
 'multiple',
 'entity',
 'system',
 'person',
 'contro',
 'token',
 'transaction',
 'wallet',
 'coin',
 

### User Defined function for generating tokens

In [22]:
def tokenization(text):
    tokens = (word_tokenize(text))
    return tokens

### User Defined function for removing stopwords

In [23]:
def remove_stopwords(text):
    words = [w for w in word_tokens if not w.lower() in stop_words]
    return words

### User Defined function for getting cosine similarity score

In [ ]:
def cosine(rvector , c):
    for i in range(len(rvector)):
        c+= l1[i]*l2[i]
    if int(c) == 0:
        return 0
    return c / float((sum(l1)*sum(l2))**0.5)

### Generate a Test dataset from multiple datasets

In [80]:
# Generate various datasets to test against
art = pd.read_csv('art_results.csv')
music = pd.read_csv('music_results.csv')
sport = pd.read_csv('sport_results.csv')
news = pd.read_csv('news_results.csv')
food = pd.read_csv('food_results.csv')
crypto = pd.read_csv('crypto_results.csv')

In [81]:
# We know the art , music , sport and news contains sentences not related to blockchain , crypto and web3
data1 = pd.DataFrame()
data1['sentence'] = art['sentence'][:300]
data1

data2 = pd.DataFrame()
data2['sentence'] = music['sentence'][:300]
data2

data3 = pd.DataFrame()
data3['sentence'] = sport['sentence'][:300]
data3

data4 = pd.DataFrame()
data4['sentence'] = news['sentence'][:300]
data4

data5 = pd.DataFrame()
data5['sentence'] = food['sentence'][:300]
data5

# We know the crypto dataset contains sentences related to blockchain , crypto and web3
data6 = pd.DataFrame()
data6['sentence'] = crypto['sentence'][:300]
data6

,sentence
0,welcome to this site was founded in may 2013 ...
1,here at we work very hard to ensure that all ...
2,each of our coin data pages has a graph that s...
3,we receive updated prices directly from many ...
4,related links new to crypto learn how to buy b...
...,...
295,the term electronic commerce refers to any fin...
296,we will refer to the storage medium as a card ...
297,a particular kind of electronic commerce is th...
298,with the rise of and the internet it is the ...


In [82]:
testData = pd.DataFrame()

testData = testData.append(data1 , ignore_index = True)
testData = testData.append(data2 , ignore_index = True)
testData = testData.append(data3 , ignore_index = True)
testData = testData.append(data4 , ignore_index = True)
testData = testData.append(data5 , ignore_index = True)
testData = testData.append(data6 , ignore_index = True)

testData['labels'] = "Disimilar"
testData['labels'][1500:] = "Similar"

testData

,sentence,labels
0,Art n 1 OED Online December 2011 Oxford Univer...,Disimilar
1,There is no generally agreed definition of wha...,Disimilar
2,The nature of art and related concepts such as...,Disimilar
3,In the of the history of art artistic works h...,Disimilar
4,Over time like Plato Aristotle Socrates and K...,Disimilar
...,...,...
1795,the term electronic commerce refers to any fin...,Similar
1796,we will refer to the storage medium as a card ...,Similar
1797,a particular kind of electronic commerce is th...,Similar
1798,with the rise of and the internet it is the ...,Similar


In [83]:
testData['predict'] = '.'

### Label the predict column based on the similarity score

In [84]:
sim = 0
dis = 0
idx = 0

for idx in range(len(testData)):
    line = testData['sentence'][idx]
    sent = tokenization(line)
    sent
    removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']

    sent = [w for w in sent if not w.lower() in stopwords.words('english')]

    # sw contains the list of stopwords
    sw = stopwords.words('english') 
    l1 =[];l2 =[]

    # remove stop words from the string
    X_set = {w for w in sent if not w in sw} 
    Y_set = {w for w in tops if not w in sw}

    # form a set containing keywords of both strings 
    rvector = X_set.union(Y_set) 
    for w in rvector:
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0)
        if w in Y_set: l2.append(1)
        else: l2.append(0)
    c = 0
    
    if cosine(rvector , c) >= 0.023:
#         if testData['labels'][idx] == 'Similar':
        testData['predict'][idx] = 'Similar'
        print(idx , testData['predict'][idx])
    else:
#         if testData['labels'][idx] == 'Disimilar':
        testData['predict'][idx] = 'Disimilar'
        print(idx , testData['predict'][idx])
        
# Print the total number of similar and dissimilar documents
print("Similar : " , sim , " Dissimilar : " , dis)

0 Disimilar
1 Disimilar
2 Disimilar
3 Disimilar
4 Disimilar
5 Disimilar
6 Disimilar
7 Similar
8 Similar
9 Disimilar
10 Disimilar
11 Disimilar
12 Disimilar
13 Disimilar
14 Disimilar
15 Disimilar
16 Disimilar
17 Disimilar
18 Disimilar
19 Disimilar
20 Disimilar
21 Disimilar
22 Disimilar
23 Similar
24 Disimilar
25 Disimilar
26 Disimilar
27 Disimilar
28 Disimilar
29 Similar
30 Disimilar
31 Disimilar
32 Disimilar
33 Disimilar
34 Disimilar
35 Similar
36 Disimilar
37 Disimilar
38 Disimilar
39 Similar
40 Disimilar
41 Disimilar
42 Disimilar
43 Disimilar
44 Disimilar
45 Disimilar
46 Similar
47 Similar
48 Disimilar
49 Disimilar
50 Disimilar
51 Similar
52 Similar
53 Disimilar
54 Similar
55 Disimilar
56 Disimilar
57 Disimilar
58 Disimilar
59 Similar
60 Similar
61 Disimilar
62 Disimilar
63 Disimilar
64 Disimilar
65 Disimilar
66 Similar
67 Disimilar
68 Disimilar
69 Disimilar
70 Disimilar
71 Disimilar
72 Disimilar
73 Disimilar
74 Disimilar
75 Disimilar
76 Disimilar
77 Disimilar
78 Disimilar
79 Disimila

616 Disimilar
617 Disimilar
618 Disimilar
619 Disimilar
620 Disimilar
621 Disimilar
622 Disimilar
623 Disimilar
624 Disimilar
625 Disimilar
626 Disimilar
627 Similar
628 Disimilar
629 Disimilar
630 Disimilar
631 Disimilar
632 Disimilar
633 Disimilar
634 Disimilar
635 Disimilar
636 Disimilar
637 Disimilar
638 Similar
639 Disimilar
640 Disimilar
641 Disimilar
642 Similar
643 Disimilar
644 Disimilar
645 Disimilar
646 Disimilar
647 Disimilar
648 Disimilar
649 Similar
650 Disimilar
651 Disimilar
652 Disimilar
653 Disimilar
654 Disimilar
655 Disimilar
656 Disimilar
657 Disimilar
658 Disimilar
659 Disimilar
660 Disimilar
661 Similar
662 Similar
663 Similar
664 Disimilar
665 Disimilar
666 Disimilar
667 Disimilar
668 Disimilar
669 Disimilar
670 Disimilar
671 Disimilar
672 Similar
673 Disimilar
674 Similar
675 Disimilar
676 Disimilar
677 Similar
678 Similar
679 Disimilar
680 Disimilar
681 Disimilar
682 Similar
683 Similar
684 Disimilar
685 Disimilar
686 Disimilar
687 Similar
688 Disimilar
689 Di

1208 Similar
1209 Disimilar
1210 Disimilar
1211 Disimilar
1212 Disimilar
1213 Similar
1214 Disimilar
1215 Disimilar
1216 Disimilar
1217 Disimilar
1218 Disimilar
1219 Disimilar
1220 Disimilar
1221 Disimilar
1222 Disimilar
1223 Disimilar
1224 Disimilar
1225 Disimilar
1226 Disimilar
1227 Disimilar
1228 Disimilar
1229 Disimilar
1230 Disimilar
1231 Similar
1232 Disimilar
1233 Disimilar
1234 Disimilar
1235 Disimilar
1236 Similar
1237 Disimilar
1238 Disimilar
1239 Disimilar
1240 Disimilar
1241 Disimilar
1242 Disimilar
1243 Disimilar
1244 Disimilar
1245 Disimilar
1246 Disimilar
1247 Disimilar
1248 Similar
1249 Disimilar
1250 Disimilar
1251 Similar
1252 Disimilar
1253 Disimilar
1254 Similar
1255 Disimilar
1256 Similar
1257 Similar
1258 Disimilar
1259 Disimilar
1260 Disimilar
1261 Disimilar
1262 Similar
1263 Similar
1264 Similar
1265 Disimilar
1266 Disimilar
1267 Similar
1268 Disimilar
1269 Disimilar
1270 Disimilar
1271 Disimilar
1272 Disimilar
1273 Disimilar
1274 Disimilar
1275 Disimilar
1276 D

1793 Disimilar
1794 Disimilar
1795 Similar
1796 Disimilar
1797 Similar
1798 Similar
1799 Similar
Similar :  0  Dissimilar :  0


### Count the number of accurate predictions

In [85]:
accuracy = 0

for idx in range(len(testData)):
    if testData['labels'][idx] == testData['predict'][idx]:
        accuracy += 1
        
accuracy

1541

In [86]:
def accuracy_score():
    return accuracy / len(testData)

In [87]:
accuracy_score()

0.8561111111111112